In [27]:
import datetime as dt
import pandas as pd
import requests
# if suppose nothing gets from url then also program will continue to run
from requests.exceptions import ConnectionError
from bs4 import BeautifulSoup

In [109]:
def webcontent_div(webcontent,classpath):
    webcontent_div=webcontent.find_all('div',{'class':classpath})
    try:
        finstreamers=webcontent_div[0].find_all('fin-streamer')
        texts=[finstreamer.get_text() for finstreamer in finstreamers]
    except IndexError:
        texts=[]
    return texts

def targetesttimate(webcontent,classpath):
    targetesttimate=webcontent.find_all('div',{'class':classpath})
    try:
        tds=targetesttimate[0].find_all('tr')
        texts=[td.get_text() for td in tds]
    except IndexError:
        texts=[]
    return texts


def realtimeprice(stockcode):
    volume=[]
    url='https://finance.yahoo.com/quote/'+stockcode+'?p='+stockcode+'&.tsrc=fin-srch'
    try:
        r=requests.get(url)
        webcontent=BeautifulSoup(r.text,'lxml')
        texts=webcontent_div(webcontent,"D(ib) Mend(20px)")
        if texts!=[]:
              price,change,changeper=texts[0],texts[1],texts[2]
        else: 
            price,change,changeper=[],[],[]
        
#         *******Volume*******
        texts=webcontent_div(webcontent,'D(ib) W(1/2) Bxz(bb) Pend(12px) Va(t) ie-7_D(i) smartphone_D(b) smartphone_W(100%) smartphone_Pend(0px) smartphone_BdY smartphone_Bdc($seperatorColor)')
        if texts!=[]:
            volume=texts[0]
        else:
            volume=''
            
#         //bearish and bullish
#         pattern=webcontent_div(webcontent,'')
#         try:
#             latest_pattern=pattern[0]
#         except IndexError:
#             latest_pattern=[]
            
#         *********1yr target estimate*******
        texts=targetesttimate(webcontent,'D(ib) W(1/2) Bxz(bb) Pstart(12px) Va(t) ie-7_D(i) ie-7_Pos(a) smartphone_D(b) smartphone_W(100%) smartphone_Pstart(0px) smartphone_BdB smartphone_Bdc($seperatorColor)')
#         print(texts)
        if texts!=[]:
           one_year_target=texts[-1][13:]
        else:
            one_year_target=''
    except ConnectionError:
        price,change,changeper,volume,one_year_target=[],[],[],'',''
    return price,change,changeper,volume,one_year_target

Stock=['BRK-B','PYPL','TWTR','AAPL','AMZN','MSFT','META','GOOG']

# print(realtimeprice('BRK-B'))
     
# ******Store values of different stocks*******
info=[]
col=[]
timestamp=dt.datetime.now()-dt.timedelta(hours=13)
timestamp=timestamp.strftime("%Y-%m-%d %H:%M:%S")
for stockcode in Stock:
    price,change,changeper,volume,one_year_target=realtimeprice(stockcode)
    info.append(price)
    info.extend([change])
    info.extend([changeper])
    info.extend([volume])
    info.extend([one_year_target])
col=[timestamp]
col.extend(info)
df=pd.DataFrame(col)
df=df.T
df.to_csv(str(timestamp[0:11])+'stockdata.csv',mode="a",header=True)
print(col)
    

['2023-02-18 11:13:30', '308.24', '+0.06', '(+0.02%)', '2,721,407', '362.00', '74.66', '-2.07', '(-2.70%)', '15,855,721', '101.47', '53.70', '0.00', '(0.00%)', '140,362,225', '44.08', '152.55', '-1.16', '(-0.75%)', '59,144,118', '169.39', '97.20', '-0.95', '(-0.97%)', '60,029,405', '135.75', '258.06', '-4.09', '(-1.56%)', '30,000,055', '285.51', '172.88', '+0.44', '(+0.26%)', '21,194,370', '212.48', '94.59', '-1.19', '(-1.24%)', '31,095,067', '123.80']
